### Enviroment 

In [134]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import json
import logging

### Crear conexion a la BD postgres

In [135]:
# Configuración del logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

try:
    # Cargar credenciales y crear motor
    with open("../credentials.json") as f:
        creds = json.load(f)

    # Crear motor y verificar conexión en un solo paso
    engine = create_engine(
        f'postgresql://{creds["user"]}:{creds["password"]}@{creds["host"]}:{creds["port"]}/{creds["database"]}'
    )
    
    # Verificar conexión (solo una vez aquí)
    with engine.connect() as conn:
        conn.execute(text("SELECT 1"))  # Consulta de prueba
        logging.info("Conexión a la base de datos exitosa")

except Exception as e:
    logging.error(f"Error de conexión: {str(e)}")
    engine = None  # Opcional: Marcar el motor como inválido

2025-03-20 21:49:05,506 - INFO - Conexión a la base de datos exitosa


In [136]:
def crear_tabla():
    with engine.begin() as connection:
        # Verificar existencia de airbnb_data
        result = connection.execute(text("""
            SELECT COUNT(*)     
            FROM information_schema.tables 
            WHERE table_name = 'airbnb_data';
        """))
        if result.scalar() != 1:
            logging.error("La tabla 'airbnb_data' no existe. Verifica que los datos hayan sido cargados correctamente.")
            return False
        
        # Eliminar airbnb_EDA si existe y crear una nueva tabla airbnb_EDA a partir de airbnb_data
        connection.execute(text("DROP TABLE IF EXISTS airbnb_EDA;"))
        connection.execute(text("""
            CREATE TABLE airbnb_EDA AS 
            SELECT * FROM airbnb_data;
        """))
        logging.info("Tabla 'airbnb_EDA' creada exitosamente a partir de 'airbnb_data'.")
        return True

def verificar_tabla():
    with engine.begin() as connection:
        # Verificar que airbnb_EDA exista
        result = connection.execute(text("""
            SELECT COUNT(*) 
            FROM information_schema.tables 
            WHERE table_name = 'airbnb_EDA';
        """))
        if result.scalar() != 1:
            logging.error("La tabla 'airbnb_EDA' no se ha creado correctamente.")
            return
        
        # Comparar cantidad de registros entre airbnb_data y airbnb_EDA
        count_original = connection.execute(text("SELECT COUNT(*) FROM airbnb_data;")).scalar()
        count_copy = connection.execute(text("SELECT COUNT(*) FROM airbnb_EDA;")).scalar()
        logging.info(f"Registros en 'airbnb_data': {count_original}")
        logging.info(f"Registros en 'airbnb_EDA': {count_copy}")

def mostrar_muestra():
    with engine.begin() as connection:
        sample = pd.read_sql("SELECT * FROM airbnb_EDA LIMIT 5;", connection)
        logging.info("Muestra de la tabla 'airbnb_EDA':")
        logging.info("\n" + sample.to_string(index=False))

if __name__ == "__main__":
    if crear_tabla():
        verificar_tabla()
        mostrar_muestra()


2025-03-20 21:49:05,663 - INFO - Tabla 'airbnb_EDA' creada exitosamente a partir de 'airbnb_data'.
2025-03-20 21:49:05,714 - INFO - Registros en 'airbnb_data': 102599
2025-03-20 21:49:05,715 - INFO - Registros en 'airbnb_EDA': 102599
2025-03-20 21:49:05,732 - INFO - Muestra de la tabla 'airbnb_EDA':
2025-03-20 21:49:05,742 - INFO - 
     id                                             NAME     host id host_identity_verified host name neighbourhood group neighbourhood      lat      long       country country code  instant_bookable cancellation_policy       room type  Construction year price service fee  minimum nights  number of reviews last review  reviews per month  review rate number  calculated host listings count  availability 365                                                                                                                                                                                                                                                                 

In [137]:
def rename_columns_with_spaces():
    """
    Renombra las columnas de la tabla 'airbnb_EDA' que contienen espacios, reemplazándolos por guiones bajos.
    """
    # Usamos engine.begin() para abrir un contexto transaccional
    with engine.begin() as connection:
        query = text("""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = 'airbnb_EDA' 
            AND column_name LIKE '% %'
        """)
        result = connection.execute(query)
        columns_to_rename = [row[0] for row in result]
        
        if not columns_to_rename:
            print("No hay columnas con espacios para renombrar.")
            return
        
        # Ejecutar las sentencias ALTER TABLE para renombrar las columnas
        for old_name in columns_to_rename:
            new_name = old_name.replace(' ', '_')
            alter_query = text(f"""
                ALTER TABLE public."airbnb_EDA"
                RENAME COLUMN "{old_name}" TO {new_name}
            """)
            connection.execute(alter_query)
            print(f"Renombrada: '{old_name}' -> '{new_name}'")

if __name__ == "__main__":
    # Llamada a la función para renombrar las columnas
    rename_columns_with_spaces()
    
    # Verificar los cambios: obtener la lista de columnas de la tabla 'airbnb_EDA'
    with engine.connect() as conn:
        query = "SELECT column_name FROM information_schema.columns WHERE table_name = 'airbnb_EDA'"
        df = pd.read_sql(query, conn)
        print("Columnas actuales en 'airbnb_EDA':")
        print(df['column_name'].tolist())


No hay columnas con espacios para renombrar.
Columnas actuales en 'airbnb_EDA':
['last_review', 'construction_year', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'id', 'host_id', 'lat', 'long', 'instant_bookable', 'NAME', 'room_type', 'host_identity_verified', 'neighbourhood_group', 'license', 'price', 'service_fee', 'cancellation_policy']


In [138]:
def eliminate_columns_innecesarias():
    """
    Elimina las columnas 'host_name', 'country_code', 'country' y 'house_rules' de la tabla 'airbnb_EDA'.
    """
    # Usamos engine.begin() para abrir un contexto transaccional
    with engine.begin() as connection:
        query = text("""
        ALTER TABLE public."airbnb_EDA"
        DROP COLUMN IF EXISTS host_name,
        DROP COLUMN IF EXISTS country_code,
        DROP COLUMN IF EXISTS country,
        DROP COLUMN IF EXISTS house_rules;
        """)
        result=connection.execute(query)
        logging.info("Se han eliminado las columnas innecesarias de 'airbnb_EDA'.")

if __name__ == "__main__":
    eliminate_columns_innecesarias()

with engine.connect() as conn:
    query = "SELECT * FROM public.\"airbnb_EDA\""
    df = pd.read_sql(query, conn)

# Mostrar solo el listado de columnas
print("Columnas de la tabla 'airbnb_EDA':")
print(df.columns.tolist())

2025-03-20 21:49:05,768 - INFO - Se han eliminado las columnas innecesarias de 'airbnb_EDA'.


Columnas de la tabla 'airbnb_EDA':
['id', 'NAME', 'host_id', 'host_identity_verified', 'neighbourhood_group', 'lat', 'long', 'instant_bookable', 'cancellation_policy', 'room_type', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'license', 'last_review']


In [139]:
def transform_last_review_to_text():
    """
    Convierte la columna 'last_review' a tipo TEXT usando una transformación explícita.
    """
    with engine.begin() as connection:
        query = text("""
            ALTER TABLE public."airbnb_EDA" 
            ALTER COLUMN last_review TYPE TEXT 
            USING last_review::TEXT;
        """)
        connection.execute(query)
        logging.info("Columna 'last_review' convertida a TEXT.")

def create_temp_last_review_column():
    """
    Crea la columna temporal 'temp_last_review' en la tabla 'airbnb_EDA'
    si esta no existe, para almacenar la conversión de fechas.
    """
    with engine.begin() as connection:
        query = text("""
            DO $$ 
            BEGIN 
                IF NOT EXISTS (
                    SELECT 1 FROM information_schema.columns 
                    WHERE table_name = 'airbnb_EDA' AND column_name = 'temp_last_review'
                ) THEN
                    ALTER TABLE public."airbnb_EDA" ADD COLUMN temp_last_review DATE;
                END IF;
            END $$;
        """)
        connection.execute(query)
        logging.info("Columna temporal 'temp_last_review' creada (si no existía).")

def update_temp_last_review_values():
    """
    Actualiza la columna temporal 'temp_last_review' basándose en el valor de 'last_review'
    para convertir distintos formatos de fecha.
    """
    with engine.begin() as connection:
        query = text("""
            UPDATE public."airbnb_EDA" 
            SET temp_last_review = 
                CASE
                    WHEN last_review ~ '[^0-9/-]' THEN NULL
                    WHEN last_review ~ '^\d{1,2}/\d{1,2}/\d{4}$' 
                        THEN TO_DATE(
                            LPAD(SPLIT_PART(last_review, '/', 1), 2, '0') || '/' ||
                            LPAD(SPLIT_PART(last_review, '/', 2), 2, '0') || '/' ||
                            SPLIT_PART(last_review, '/', 3),
                            'MM/DD/YYYY'
                        )
                    WHEN last_review ~ '^\d{4}-\d{2}-\d{2}$' 
                        THEN TO_DATE(last_review, 'YYYY-MM-DD')
                    ELSE NULL 
                END;
        """)
        connection.execute(query)
        logging.info("Columna 'temp_last_review' actualizada con valores convertidos desde 'last_review'.")

def update_null_temp_last_review():
    """
    Reemplaza los valores NULL en 'temp_last_review' por la fecha '1900-01-01'.
    """
    with engine.begin() as connection:
        query = text("""
            UPDATE public."airbnb_EDA"
            SET temp_last_review = '1900-01-01'
            WHERE temp_last_review IS NULL;
        """)
        connection.execute(query)
        logging.info("Valores nulos en 'temp_last_review' actualizados a '1900-01-01'.")

def drop_last_review_and_rename_temp():
    """
    Elimina la columna original 'last_review' y renombra 'temp_last_review' a 'last_review'.
    """
    with engine.begin() as connection:
        query = text("""
            ALTER TABLE public."airbnb_EDA" DROP COLUMN last_review;
            ALTER TABLE public."airbnb_EDA" RENAME COLUMN temp_last_review TO last_review;
        """)
        connection.execute(query)
        logging.info("Columna 'last_review' eliminada y 'temp_last_review' renombrada a 'last_review'.")

def transform_last_review_to_integer():
    """
    Convierte la columna 'last_review' (ahora de tipo DATE) a INTEGER en formato AAAAMMDD.
    """
    with engine.begin() as connection:
        query = text("""
            ALTER TABLE public."airbnb_EDA" 
            ALTER COLUMN last_review TYPE INTEGER 
            USING (
                EXTRACT(YEAR FROM last_review) * 10000 +
                EXTRACT(MONTH FROM last_review) * 100 +
                EXTRACT(DAY FROM last_review)
            );
        """)
        connection.execute(query)
        logging.info("Columna 'last_review' convertida a INTEGER en formato AAAAMMDD.")

def execute_sql_transformations():
    """
    Ejecuta todas las transformaciones de fechas y el manejo de valores nulos en la tabla 'airbnb_EDA'.
    """
    transform_last_review_to_text()
    create_temp_last_review_column()
    update_temp_last_review_values()
    update_null_temp_last_review()
    drop_last_review_and_rename_temp()
    transform_last_review_to_integer()
    logging.info("Transformación de fechas y manejo de valores nulos completado correctamente.")


In [141]:
query = 'SELECT * FROM public."airbnb_EDA";'
with engine.connect() as connection:
    df = pd.read_sql(query, connection)
    print(df.head())

print("\n===============================================")

query = 'SELECT COUNT(*) FROM public."airbnb_EDA"'
with engine.connect() as connection:
    count = pd.read_sql(query, connection)
    print(count)


         id                NAME      host_id host_identity_verified  \
0   1112901                None  21389589535            unconfirmed   
1   1150457                None  87092192250               verified   
2   4109680     1 Room & 1 Bath  32147733106               verified   
3   9688463  Private Bedroom #2  66554202396               verified   
4  10710218                room  96659601480               verified   

  neighbourhood_group       lat      long instant_bookable  \
0              Queens  40.71546 -73.87854             None   
1            Brooklyn  40.68317 -73.94701             True   
2            Brooklyn  40.69216 -73.90859             True   
3              Queens  40.74776 -73.88827             True   
4           Manhattan  40.80079 -73.96619            False   

  cancellation_policy        room_type  ...    price service_fee  \
0                None  Entire home/apt  ...    $907        $181    
1              strict     Private room  ...    $343         $69 

In [142]:
with engine.begin() as connection:
    connection.execute(text("""
        UPDATE public."airbnb_EDA"
        SET neighbourhood_group = CASE 
            WHEN neighbourhood_group = 'brookln' THEN 'Brooklyn'
            WHEN neighbourhood_group = 'manhatan' THEN 'Manhattan'
            ELSE neighbourhood_group
        END
        WHERE neighbourhood_group IN ('brookln', 'manhatan');
    """))
    
print("Datos actualizados correctamente en la columna 'neighbourhood_group'.")


Datos actualizados correctamente en la columna 'neighbourhood_group'.


In [143]:
df = pd.read_sql('SELECT DISTINCT neighbourhood_group FROM public."airbnb_EDA"', engine)
df

,neighbourhood_group
0,Bronx
1,None
2,Brooklyn
3,Queens
4,Staten Island
5,Manhattan
